In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xml.etree.ElementTree as ET
import os
from tifffile import imread
import shutil

from vtk_to_tiff_py import save_imagej_tiff, save_imagej_tiff4d

#deta = np.genfromtxt('C:/Users/bioAMD/Desktop/Nathan/Tracking/070418a_0.emb', skip_header=1, delimiter=";")

 Fichiers EMB 
 
0 : Global Id cell (Id de la cellule traquée)

1 : Local Id cell (inutile)

2, 3, 4: Coordonnées x, y, z en PIXELS

5 : Pas de temps

6 : Mother local Id (inutile)

7 : Validation -> -1: pas vérifié

                   0: vérifié et faux
                   
                   1: vérifié et vrai
                   
                   2: vérifié mais non traquée (vrai) 
                   
8: Mother global Id cell (Id de la cellule mère, cellule traquée au temps t - 1)

In [4]:
#positions = np.copy(deta[:,2:5])
#print(positions.shape)
#cond = np.zeros(tuple([3]) + positions[:,0].shape)
#print(cond.shape)
#for i in range(len(cond)) :
#    if i != 2 :
#        cond[i,:] = np.logical_and( positions[:,i] < 256, positions[:,i] > -256)
#    else :
#        cond[i,:] = positions[:,i] < 104
#    
#    cond = cond.astype(np.bool_)
#    positions[ np.logical_and(cond[0], cond[1], cond[2]) ]
#
#i = 2
#plt.hist(positions[:,i], bins=30)
#print(len(positions))

In [5]:
def produit_scalaire(vec1, vec2) :
    assert ( len(vec1.shape) == 1 and len(vec2.shape) == 1), "Pas des vecteurs."
    np.sum(vec1*vec2)
    
def norme(vec) :
    return np.sqrt(np.sum(vec**2))

def xml_to_dataframe(dir_xml) :
    
    dict_data = dict()
    # on lit les données du fichier XML sous forme d'arbre
    tree = ET.parse(dir_xml)
    root = tree.getroot() # Experiment
    for i,key in enumerate(root.attrib) : # on parcourt les éléments de Experiment
        try :
            dict_data[key] = int(root.attrib[key])
        except :
            dict_data[key] = root.attrib[key]
            
    return pd.DataFrame.from_dict(dict_data, orient="index")

In [6]:
def clean_data(data, metadata_dir) :
    """ filtre les données ayant mal été traquées par l'algorithme.
    data : données du fichier EMB 
    metadata_dir : Métadonnées du fichier XML """
    
    metadata = xml_to_dataframe(metadata_dir)
    data_final = np.copy(data)
    
    # distance entre frontière de la boîte et le centre (1/2 x longueur)
    l = np.zeros(3)
    for i, axe in enumerate(['X', 'Y', 'Z']) :
        if axe == "Z" :
            l[i] = int(metadata.loc['db_nbPixel'+axe])
        else:
            l[i] = int(metadata.loc['db_nbPixel'+axe])//2
                
    # on ne conserve que les cellules à l'intérieur de la boîte
    cond = np.empty( tuple([3]) + data_final[:,0].shape)
    for i in range(len(cond)) :
        if i != 2 :
            cond[i] = np.logical_and(data_final[:,i+2] <= l[i], data_final[:,i+2] > -l[i])
        
        # l'axe commence à 0
        else :
            cond[i] = np.logical_and( data_final[:,i+2] <= l[i], data_final[:,i+2] >= 1)
        
    cond = cond.astype(np.bool_)
    data_final = data_final[np.logical_and(np.logical_and(cond[0], cond[1]), cond[2])]
    
    # on enlève les données classées comme fausses 
    data_final = data_final[ data_final[:,7] != 0 ]   
    
    # on ajoute les longueurs aux coordonnées afin d'éviter les valeurs négatives
    for i, axe in enumerate(['X', 'Y', 'Z']) :
        if axe != 'Z':
            L = int(metadata.loc['db_nbPixel'+axe])//2 -1
        else :
            L = -1
        data_final[:,i+2] += L
    
    return data_final

#mdata_dir = "C:/Users/bioAMD/Desktop/movit941/MovitData/local/070418a/070418a_0.xml"
#data = clean_data(deta, mdata_dir)
#print(data.shape)

In [7]:
def patch(array, vecteur, taille_patch: tuple[int,...], array4d=False, axis=0) :
    """ Crée un patch autour d'un point de coordonnées vecteur (en pixels). 
    axis : indice de l'axe que l'on garde en totalité 
    vecteur : array 1D à 3 composantes entières positives """
    
    patch_size = np.uint16(taille_patch)
            
    # on s'assure qur le patch ne dépasse pas des bords de l'image 
    axe_skip = 0
    for i in range(len(array.shape)) :
        if i == axis and array4d :
            axe_skip += 1
            continue 
            print(i, np.ma.size(array, axis=i))
        assert (vecteur[i-axe_skip] - patch_size[i-axe_skip]//2 >=0 
                and vecteur[i-axe_skip] + patch_size[i-axe_skip]//2 < np.ma.size(array, axis=i) ), "le patch dépasse des bords"
    
    assert axis < 4, "axis trop grand."
    assert len(vecteur) == len(patch_size) #and len(patch_size) == 3
    
    if array4d :
        if axis == 1 :
            newArray = array[vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,:,
                            vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                            vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2 ]       
        elif axis == 2 :
            newArray = array[vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,
                            vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                            :, vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2 ] 
        elif axis == 3 :
            newArray = array[vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,
                            vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                            vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2, : ]
        else :
            newArray = array[ :, vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,
                            vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                            vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2 ]
    else :
        newArray = array[vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,
                           vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                           vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2 ] 
    return newArray

def positions(emb_data, t, clean=False, metadata_dir=None) :
    """ retourne les positions des cellules au temps t"""
    
    assert t <= np.max(emb_data[:,5]), "t trop grand"
    
    data = emb_data
    if clean :
        data = clean_data(data, metadata_dir)
    data = data[np.argwhere(data[:,5] == t)[:,0], 2:5]
    return data

def cellule_temps(globalId, emb_data, stop= None, clean=False, metadata_dir=None) :
    """ Suit une cellule au cours du temps, à l'envers.
    
    globalId : Id de la cellule au pas de temps choisi. 
    clean : Nettoie les données avant de procéder à l'aide du fichier xml (metadata_dir) et de clean_data 
    stop: nombre de pas revenus en arrière. len(positions) = stop + 1
    
    Retourne :
        positions : tableau (x, y, z) de la cellule à chaque pas de temps (en pixels)
        temps : Pas de temps où le tracking a fonctionné 
        ids_cellule  : GlobalIds pris par la cellule à chaque pas de temps """
    
    assert len(np.argwhere(emb_data[:,0] == globalId)) != 0, "Cellule {} non trouvée".format(globalId)
    
    data = emb_data
    if clean :
        data = clean_data(data, metadata_dir)
    
    data_init = data[ data[:,0] == globalId][0]
    #print(data_init)
    
    # initialisation
    positions = np.zeros(shape=(int(data[-1,5]), 3))
    positions[0], motherId = data_init[2:5], data_init[-1]
    
    # Pas de temps où le tracking fonctionne
    temps = np.zeros(int(data[-1,5]))
    temps[0] = 0
    
    # globalIds de la cellule au cours du temps 
    ids_cellule = np.zeros(int(data[-1,5]))
    ids_cellule[0] = motherId
    
    for j in range(1,len(positions[:,0])) :       
        
        # cellule dont le global Id est la mère de la cellule précédente
        mother = data[data[:,0] == motherId]
        if motherId != 0 and len(mother) > 0 :
            mother = mother[0]
        else :
            mother = []
        
        # indice de la cellule précédente ayant une mère : tracking réussi
        if len(mother) != 0 :
            cell_temp = mother[0]
        else :
            break
        
        # on extrait la position dela cellule mère et le pas de temps associé
        positions[j] = mother[2:5]
        temps[j] = mother[5]
        ids_cellule[j] = mother[0]
            
        # id de la cellule mère de la cellule mère
        motherId = mother[-1]
        
        #if motherId == 0 and temps[j] != 0 :
        #    #print("Cellule {} n'a pas de mère : tracking échoué".format(str(int(cell_temp))))
        #elif motherId == 0 and temps[j] == 0 :
            #print('Cellule {} bien traquée.'.format(str(int(cell_temp))))
            
        # on arrête tout si on dépasse la limite fixée
        if stop != None and len(positions[:j]) >= stop :
            break
    
    # on sélectionne uniquement les pas de temps où on peut tracer le linéage de la cellule
    trackingSuccess = np.argwhere(positions != np.zeros(3))[::3,0]
    
    return positions[trackingSuccess], temps[trackingSuccess], ids_cellule[trackingSuccess]

def position_cellule(data, globalId: int, t: int, divdata=[]) :
    """ retourne la position de la cellule globalId au pas de temps t """
    
    if len(divdata) == 0 :
        mask = np.logical_and( data[:,0] == globalId, data[:,5] == t )
        assert len(data[mask]) != 0, "Cellule {} pas trouvée au pas de temps t = {}.".format(str(int(globalId)), t)
        return data[mask, 2:5]
    else :
        mask = np.logical_and( divdata[:,1] == globalId, divdata[:,0] == t)
        assert len(divdata[mask]) != 0, "Cellule {} pas trouvée au pas de temps t = {}.".format(str(int(globalId)), t)
        return divdata[mask, 2:5]

def cellules_divisions(data, t) :
    """donne les globalID des cellules qui se divisent au temps t et ses filles. 
    Une cellule se divise au temps t lorsqu'au pas de temps t+1, elle a deux filles """
    
    cellules_id = data[ data[:,5] == t ]
    
    # pas de temps d'après 
    temp_filles = data[ data[:,5] == t+1 ]
    
    # liste des globalId des mères qui se divisent
    divisions = []
    
    assert len(cellules_id) > 0, "Aucune cellule à ce pas de temps"
    
    for motherId in cellules_id[:,0] :

        filles = temp_filles[ temp_filles[:,-1] == motherId, 0 ]
        
        # si il y a division cellulaire 
        if len(filles) > 1 :
            divisions.append(motherId)
            
    return divisions

def save_divisions(data, save_dir, tstart=0, tstop=None, erase=False) :
    """ sauvegarde dans un fichier les positions des cellules qui se divisent au cours du temps """
    
    position = np.zeros(3)
    
    # si le fichier de sauvegarde existe déja, on écrit par dessus (sauf si erase est vrai)
    if os.path.exists(save_dir) :
        if erase :
            f = open(save_dir, 'w')
        else :
            f = open(save_dir, 'a')
    else :
        f = open(save_dir, 'w')
        
    f.write('# t globalId x y z \n')
    f.close()
    
    # tableau des pas de temps
    tstop_init = 0
    if tstop == None :
        tstop_init = np.max(data[:,5])
        assert tstart <= np.max(data[:,5]) and tstart >= 0, "Mauvais intervalle de temps."
    else :
        tstop_init = tstop
        assert tstop <= np.max(data[:,5]) and tstop>= tstart, "Mauvais intervalle de temps."
    temps = np.arange(tstart, tstop_init+1, step=1)
    
    for t in temps :
        
        f = open(save_dir, 'a')
        
        # on parcourt les cellules qui se divisent et on extrait leur position
        divisionsId = cellules_divisions(data, t)
        
        for i in range(len(divisionsId)) :
            
            f.write(str(t) + " ")
            f.write(str(int(divisionsId[i])) + " ")
            
            position = position_cellule(data, divisionsId[i], t)[0]
            
            # on écrit les 3 composantes de chaque cellule sur la même ligne
            for j in range(len(position)) :
                f.write(str(position[j]) + " ")
            
            f.write("\n")
        
        f.close()
                
        # nouveau pas de temps: on change de ligne 
        print("t =",t, "/", np.max(temps))
        
        # on sauvegarde à chaque pas de temps
        
    return None

def filles_division(data, t, deltat, divfile="") :
    """ Calcule les Id des filles de cellules qui se divisent au pas de temps t.
    deltat : nombre de pas de temps après la division pour lequel on trouve l'ancêtre
    from_file : si on dispose du fichier avec toutes les divisions 
    divfile : Répertoire où se trouve le fichier des divisions 
    
    Retourne :
        dict_liens: dictionnaire où chaque clé correspond au globalId d'une mère (au temps t), et chaque valeur l'ensemble
                    des globalId des filles associées (au pas de temps t+deltat)
        """
    print('fonction FILLES_DIVISION')
    
    # cellules au pas de temps t+deltat
    cellules_fin = data[ data[:,5] == t+deltat, 0]
    
    # cellules s'étant divisées au pas de temps t
    if divfile == "" :
        cellsdiv = cellules_divisions(data, t)
    else :
        datadiv = np.genfromtxt(divfile, skip_header=1)
        cellsdiv = datadiv[datadiv[:,0] == t, 1]
    
    assert len(cellules_fin) > 0, "Aucune cellule à ce pas de temps."
    assert len(cellsdiv) > 0, "Aucune division à ce pas de temps."
    
    # on répertorie les mères au pas de temps t
    set_cellsdiv = set()
    dict_liens = {}
    for motherId in cellsdiv :
        dict_liens[motherId] = set()
        set_cellsdiv.add(motherId)
    
    # on parcourt les cellules du pas de temps t + deltat
    compteur = 0
    for filleId in cellules_fin :
        
        positions, temps, ids = cellule_temps(filleId, data, stop=deltat)
        if len(positions) <= deltat :
            print( "Tracking échoué : cellule mère de {} perdue ({})".format(str(int(ids[0])), len(positions)) )
            continue 
        
        # si la cellule mère est une des cellules qui s'est divisée au pas de temps t,
        # on rajoute la fille sur le dico avec la clé de la mère 
        if ids[deltat] in set_cellsdiv :
            print("   BINGO :", ids[deltat], compteur)
            dict_liens[ ids[deltat] ].add(filleId)
            
        if 100* compteur / (len(cellules_fin)-1) % 5 == 0 :
            print("filles_division: ",100* compteur / (len(cellules_fin)-1),'%')
        compteur += 1
                
    return dict_liens

def calcul_deplacements(data, dict_liens, t, deltat, divfile="") :
    """ Calcule les vecteurs reliant les filles (temps t+deltat) lors d'une division cellulaire au pas de temps t
    
        dict_liens : dict{motherId: {filles},...}
        deltat: nombre de pas où l'on revient en arrière. 
        divfile: Répertoire du fichier des divisions cellulaires
    Retourne :
        vect_division: vecteurs reliant les deux filles entre elles pour chaque cellule mère
        positions_meres: vecteur position pour chaque cellule mère.
        
    """
    
    if divfile != "" :
        datadiv = np.genfromtxt(divfile, skip_header=1)
    else : 
        datadiv= []
        
    vect_division = np.zeros((len(dict_liens), 3))
    positions_meres = np.zeros((len(dict_liens), 3))
    
    #vecteurs unitaires 
    e = np.identity(3)
    
    # on parcourt chaque cellule mère et on regarde la position de ses 2 filles
    for i,keyId in enumerate(dict_liens) :
        
        filles = list(dict_liens[keyId])
        if len(filles) <= 1 :
            print( 'Deuxième fille de {} perdue. '.format(str(int(keyId))) )
            continue
        
        # vecteur reliant les deux filles (axes XYZ)
        position1 = position_cellule(data, filles[0], t+deltat)
        position2 = position_cellule(data, filles[1], t+deltat)
            
        vect_division[i] = position2 - position1
        
        #position de la mère
        positions_meres[i] = position_cellule(data, keyId, t, datadiv)
        
    return vect_division, positions_meres

def arrondi(nbre: float) :
    """ arrondit à l'entier le plus proche """
    if np.abs(nbre//1 - nbre) <= np.abs((nbre+1)//1 - nbre) :
        ent = nbre//1
    else :
        ent = nbre//1 + 1
    return int(ent)

def patch_gaussien(vecteur, array_size, patch_size: tuple[int,...], covmat, add=False, img=None) :
    """ Retourne un patch densité de probabilité gaussienne dans un espace à 3 dimensions.
        Longueurs en PIXELS
        
    vecteur : centre du patch, vecteur 3D
    covmat : Matrice de covariance de la distribution
    add : On ajoute les valeurs sur une matrice 3D déja existante (img)
    """
    
    # sécurité
    if not add :
        array = np.zeros(array_size)
    else :
        array = np.copy(img)

    # fonction gaussienne avec 256 pour maximum 
    g = lambda x : arrondi( 256*np.exp(-(x - vecteur).T@np.linalg.inv(covmat)@(x - vecteur)) )
    
    # vecteur déplacement effectué par la transformation array -> patch
    deplacement = np.zeros(len(array_size))
    for i in range(len(deplacement)) :
        deplacement[i] = vecteur[i] - patch_size[i]//2
    
    # on parcourt les indices de chaque élément du patch, et on calcule la distance avec vecteur (son centre)
    
    cpte = 0
    for i in range(patch_size[0]) :
        for j in range(patch_size[1]) :
            for k in range(patch_size[2]) :
                indx, indy, indz = int(i + deplacement[0]), int(j + deplacement[1]), int(k + deplacement[2])
                
                if indx < 0 or indx >= len(array[:,0,0]) :
                    cpte += 1
                    continue
                if indy < 0 or indy >= len(array[0,:,0]) :
                    cpte += 1
                    continue
                if indz < 0 or indz >= len(array[0,0,:]) :
                    cpte += 1
                    continue
                x = np.array([indx, indy, indz])
                array[indx, indy, indz] += g(x)
                
                if array[indx, indy, indz] > 255 :
                    array[indx, indy,indz] = 255
    print(cpte)
                
    return array

def generate_labels(data, array_size, patch_size, t, deltat, sigma, divfile="") :
    """Génère des images ne comportant que les cellules se divisant au pas de temps t 
    
        deltat : nombre de pas de temps avancés pour identifier les filles
        array_size : taille de l'image generée
        patch_size : taille du patch sur lequel on ajoute les valeurs de la gaussienne
        sigma : variance des directions principales 
        divfile : Répertoire du fichier des divisions cellulaires. shape=(t, globalId, x,y,z)
                    
    Retourne :
        image : array 3D en sortie 
    """

    # dictionaire repertoriant les Id des filles de chaque cellule s'étant divisée
    dict_liens = filles_division(data, t, deltat, divfile)
    
    # vecteurs reliant les filles et positions des mères 
    vect_deplacement, positions_meres = calcul_deplacements(data, dict_liens, t, deltat, divfile)
    
    # on modifie les axes des positions mères : axes XYZ -> ZYX 
    temp = np.copy(positions_meres[:,2])
    positions_meres[:,2] = positions_meres[:,0] # XYZ -> XYX
    positions_meres[:,0] = temp # XYX -> ZYX
    
    # et ceux des vecteurs de déplacement
    temp = np.copy(vect_deplacement[:,2])
    vect_deplacement[:,2] = vect_deplacement[:,0] # XYZ -> XYX
    vect_deplacement[:,0] = temp # XYX -> ZYX
    
    # on enlève les 0 des positions des mères et des vecteurs déplacement
    filtre = np.argwhere( positions_meres != np.zeros(len(array_size)))[::len(array_size),0]
    vect_deplacement = vect_deplacement[filtre]
    positions_meres = positions_meres[filtre]
    
    # image finale
    image = np.zeros(array_size)
    
    for i in range(len(vect_deplacement)) :  
        
        # on modifie la matrice de covariance pour prendre en compte l'étalement des cellules qui se divisent  
        matcov = np.identity(len(array_size))*sigma #+ np.random.random((len(array_size), len(array_size)))*sigma/4
        
        vecteur = vect_deplacement[i]
        
        for j in range(3) :
            matcov[j,j] += min(np.abs(vecteur[j]), sigma)
        
        # on ajoute le patch gaussien generé à l'image finale
        image = patch_gaussien(positions_meres[i], array_size, patch_size, matcov, add=True, img = image)
        
    return image.astype(np.uint16)

In [8]:
# savedir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/070418a_divisions.txt"
# save_divisions(data, savedir, tstart=0, tstop=500, erase=True)

In [9]:
########### Créattion des patch gaussiens ##################
#base = "C:/Users/bioAMD/Desktop/Nathan/Tracking/patch_gaussiens"
#div_fichier="C:/Users/bioAMD/Desktop/Nathan/Tracking/070418a_divisions.txt"
#
#scale = tuple(np.float32([1.34, 1.34, 1.34]))
#array_size = (104, 512, 512)
#patch_size = (24, 38, 38)
#
#for t in range(304, 500) :
#    image_verdict = generate_labels(data, array_size, patch_size, t, deltat=2, sigma=5, divfile=div_fichier)
#    nomFichier = "070418a_t"+str(t).zfill(3)+"_ch09.tif"
#    save_imagej_tiff(base+"/"+nomFichier, image_verdict, scale, units="um")
#    print(t, "/", 500)

In [10]:
def patch_label(date, images_dir, nb_patches, patch_size, t, maxIter, divfile, array4d=False) :
    """Crée des patch (tirés aléaoirement) à partir d'une image 3D des labels. 
    Ils contiennent au moins une division cellulaire.
    
    date : jeu de donnéees utilisé
    images_dir: répertoire des images 3D labels (patch gaussiens)
    nb_patches: nombre de patches tirés 
    maxIter: Nombre maximal de tours de boucles par pas de temps
    divfile : répertoire du fichier des divisions cellulaires
    
    Retourne: 
        patches_div : array4D shape = (nb_patches, patch_size) des labels
        vecteurs_div: coordonnées des centres (aléatoires) des patch contenant des divisions
        vecteurs_nodiv : coordonnées de patch ne contenant pas de divisions
        indice_t : renvoie l'indice du fichier correspondant au temps t dans os.listdir(images_dir)
        
        """
    
    indice_t = None
    
    # on sélectionne l'image correspondant au temps t
    files = os.listdir(images_dir)
    file_t = ""
    for i, file in enumerate(files) :
        s = 0
        while file[len(date)+2+s] != "_" :
            s += 1
        if int(file[len(date)+2:len(date)+2+s]) == t and file[:len(date)] == date :
            file_t = file
            indice_t = i
            
    assert file_t != "", 'Pas de temps t = {} non trouvé dans {}.'.format(t, images_dir)
    
    # image 3D à partir du tif
    image = imread(images_dir+"/"+file_t)
        
    # positions de toutes les divisions 
    datadiv = np.genfromtxt(divfile, skip_header=1)
    datadiv = datadiv[datadiv[:,0] == t, 2:5]
    
    # on s'assure que l'image correspond bien au pas de temps t
    for i in range(len(datadiv)) :
        l,j,k = tuple(datadiv[i].astype(np.uint16))
        if image[k,j,l] < 253 :
            print(image[k,j,l], (k,j,l))
            
            # si il y a une erreur, patch_label ne renvoie que des 0.
            return 0,0,0,0
            #raise Exception("L'array 3D ne correspond pas au pas de temps t.")
    
    patches_div = np.zeros(tuple([nb_patches]) + patch_size)
    vecteurs_div = np.zeros(shape=(nb_patches, 3))
    vecteurs_nodiv = []
    indices_nodiv = []
    
    assert maxIter > nb_patches, "maxIter < nombre de patches. Augmenter maxIter."
    
    # prévention boucle infinie
    iter_count = 0
    
    # nombre de patches sauvegardés
    nb_div = 0
    
    while nb_div < nb_patches and iter_count < maxIter :
        
        # vecteur 3D aléatoire 
        low_indices = [patch_size[i]//2 for i in range(len(patch_size)) ]
        high_indices = [ np.ma.size(image, axis=i) - patch_size[i]//2 for i in range(len(patch_size)) ]
        
        # vecteur aléatoire 
        vecteur = np.random.randint(low=low_indices, high=high_indices, size=3)
        
        # prévention des dépassements de bords
        for i in range(len(patch_size)) :
            if vecteur[i] - patch_size[i]//2  < 0 or vecteur[i] + patch_size[i]//2 > np.ma.size(image, axis=i) :
                raise Exception("Erreur: patch déborde de l'image sur l'axe {}.".format(i))
        
        patch = image[vecteur[0]-patch_size[0]//2:vecteur[0]+patch_size[0]//2,
                      vecteur[1]-patch_size[1]//2:vecteur[1]+patch_size[1]//2,
                      vecteur[2]-patch_size[2]//2:vecteur[2]+patch_size[2]//2 ]
    
        # si le patch contient une division cellulaire, on le sauvegarde 
        if np.any(patch > 252) :
                patches_div[nb_div] = patch
                vecteurs_div[nb_div] = vecteur
                nb_div += 1
        else :
            vecteurs_nodiv.append(vecteur)
            
        iter_count += 1
        
    return patches_div, vecteurs_div, np.array(vecteurs_nodiv).astype(np.uint16), indice_t


In [11]:
def sort_timesteps(date:str, files_dir:str, formats=".tif") :
    
    files = os.listdir(files_dir)
    # on enlève les fichiers non concernés
    for file in files :
        if file[-len(formats):] != formats or file[:len(date)] != date :
            files.remove(file)
            continue

    # on répertorie tous les pas de temps
    indexes = np.zeros(len(files))
    for i, file in enumerate(files) :
        s = 0
        while file[len(date)+2+s] != "_" :
            s += 1
        indexes[i] = int(file[len(date)+2:len(date)+2+s])
    
    # on applique la permutation de argsort à la liste de fichiers
    mask = np.argsort(indexes)
    files = np.array(files)
    return files[mask]

   
    
def load_samples(date, input_tifdir, fichiers, indt, demistep=True, troispas=False, nb_forward=1 ) :
    """ Retourne les 2*nb_forward + 1 images 3D correspondant aux pas (t-nb_forward,...,t+nb_forward) sous forme
    d'un tableau 4D (3D si troistep est faux)
    
    fichier : liste des noms de fichiers du répertoire choisi
    input_tifdir: répertoire contenant les tif des images de noyaux 
    indt : indice dans la liste des fichiers input de l'image que l'on charge 
    """
    files = sort_timesteps(date, input_tifdir, formats=".tif")
    
    if troispas :
        assert indt >= nb_forward and indt + nb_forward < len(files)
        
    ### on charge les tif correspondant au pas de temps t ou aux pas de temps t-1,t,t+1
    # cas où l'image contient deux canaux
    shape_data = imread( input_tifdir + "/" +files[indt]).shape
    if troispas and demistep :
        X = np.zeros(tuple([shape_data[0]]) + tuple([2*nb_forward+1]) + shape_data[2:])
        
        # on choisit le premier canal et on ne lit que les temps paires 
        for i in range(2*nb_forward+1) :
            X[:,i,:,:] = imread(input_tifdir + "/" +files[2*(i+indt-nb_forward-1)])[:,0,:,:]
            print(input_tifdir + "/" +files[2*(i+indt-nb_forward-1)])
            
    # cas où on veut plusieurs pas mais avec des images à 1 canal
    elif troispas :
        X = np.zeros( shape_data[0] +tuple([nb_forward]) + shape_data[1:])
        
        # on prend les pas de temps dans { t-nb_forward,...,t+nb_forward }
        for i in range(2*nb_forward+1) :
            X[:,i,:,:] = imread( input_tifdir + "/" +files[i+indt-nb_forward-1])
    else :
        X = imread(input_tifdir + "/" +files[indt])
    
    return X

def save_samples_patch(date, input_tifdir, gausslabels_dir, output_dir, scale, nb_patches_image, patch_size, divfile,
              tstart, tstop, tstep, ch, demistep=False, troispas=False, nb_forward=1):
    
    """ Sauvegarde les labels et les images associées dans un répertoire train_dir. 
    ATTENTION : fichiers sauvegardés -> p = paire (div) et p = impaire (nodiv)
    
    input_tifdir : répertoire des images tif des cellules 
    output_dir : répertoire des données d'entrainement. Contient deux dossiers (samples et labels)
    gausslabels_dir : répertoire contenant les tif les labels (points gaussiens)
    
    nb_patches_image : nombre de patches génerés pour chaque image
    divfile : fichier des divisions cellulaires. Si il n'y en a pas, mettre divfile = ""
    demistep : les tifs dans input_tifdir sont des données à deux canaux (bool)
    troispas : bool qui indique si on prend plusieurs pas pour les données d'entrainement
    nb_forward : nombre de pas pris (nb channels = 2*nb_forward + 1)
    
    Retourne :
        tif des données d'entrainement (samples) rangés dans output_dir/trains_samples
        ind_skip : liste des pas de temps passés pour cause d'erreur.
        
    """
    files_samples = os.listdir(input_tifdir)
    ind_skip = []
        
    # nb d'itérations maximal pour chaque pas de temps
    maxIter = 1000
    
    if troispas :
        assert tstart >= nb_forward
    
    for t in range(tstart, tstop, tstep) :
        
        patches_div, vecteurs_div, vecteurs_nodiv, indt = patch_label(date, gausslabels_dir, nb_patches_image, 
                                                                      patch_size, t, maxIter, divfile, array4d=demistep)
        
        print("t=", t)
        
        # si patch_label ne retourne que des 0, on passe le pas de temps t
        if isinstance(patches_div, int) : 
            print(patches_div)
            ind_skip.append(t)
            continue 
        assert len(vecteurs_nodiv) >= len(vecteurs_div), "vecteurs_nodiv: pas assez d'éléments"
        
        
        ### on charge les tif correspondant au pas de temps t ou aux pas de temps t-1,t,t+1
        X = load_samples(date, input_tifdir,files_samples, indt, demistep, troispas, nb_forward)
            
        # on sauvegarde les patch des images d'entrée
        for i in range(len(vecteurs_div)) :
                        
            # patch contenant ou non une division cellulaire
            X_div = patch(X, vecteurs_div[i].astype(np.uint16), patch_size, array4d=demistep, axis=1)
            X_div = X_div.astype(np.uint16)
            
            # nom des fichiers de sauvegarde
            sample_div = output_dir + "/train_samples/" + date + "_t" + str(t).zfill(3) + "p" + str(2*i).zfill(2) + "_ch" + str(ch).zfill(2) + ".tif"
            
            label_div =  output_dir + "/train_labels/" + date + "_t" + str(t).zfill(3) + "p" + str(2*i).zfill(2) + "_ch" + str(ch).zfill(2) +".tif"
            
            # sauvegarde des samples avec et sans divisions 
            save_imagej_tiff4d(sample_div, X_div, scale, units="um")
            
            # sauvegarde des labels avec et sans divisions
            save_imagej_tiff(label_div, patches_div[i].astype(np.uint16), scale, units="um")
            
        print(t, "/", tstop-1)  
    return ind_skip

def data_rotations(tif_dir, output_dir, axes_flipped, scale, units, labels_dir="", labels_output="") :
    """ Augmente le jeu de données en inversant l'ordre des éléments de chaque array dans tif_dir selon les 
    axes définis dans axes_flipped.
    Fichier de sortie: 070418a_t002_ch06p34c01_ch05.tif """
    
    files = os.listdir(tif_dir)
    if labels_dir != "" :
        labels = os.listdir(labels_dir)
        
    for i, filename in enumerate(files) :
        
        # nom du fichier de sortie (on enlève chxx.tif)
        ch = filename[-6:-4]
        
        for axe in axes_flipped :
            
            # on charge l'array correspondant au tif 
            X = imread(tif_dir+"/"+filename).astype(np.uint16)
            
            # si l'axe est None, on retourne le fichier tel quel
            if axe == None :
                save_imagej_tiff4d(output_dir+"/"+filename, X, scale, units)
                if labels_dir != "" :
                    Y = imread(labels_dir+"/"+labels[i]).astype(np.uint16)
                
                    # on sépare les cas où il y a plusieurs channels pour les labels
                    if len(Y.shape) == 3 :
                        save_imagej_tiff(labels_output+"/"+labels[i], Y, scale, units)
                    else :
                        save_imagej_tiff4d(labels_output+"/"+labels[i], Y, scale, units)
                
                continue 
            
            # sinon, on modifie un des axes 
            X = np.flip(X, axis= axe).astype(np.uint16)
            newname = filename[:-9] + 'f'
            
            # si il y a plusieurs axes à flip, on les écrit à la suite
            if isinstance(axe, tuple) :
                for k in range(len(axe)) :
                    newname += str(axe[k])  
            else  :
                newname += str(axe).zfill(2)
                
            newname += '_ch' + ch + ".tif"
            
            # si on fournit le dir des labels, on modifie aussi ces données là et on les sauvegarde
            if labels_dir != "" :
                Y = imread(labels_dir+"/"+filename)
                if len(Y.shape) == 4 :
                    Ynew = np.flip(Y, axis= axe)
                    save_imagej_tiff4d(labels_output+"/"+newname, Ynew, scale, units)
                elif len(Y.shape) == 3 :
                    # on rajoute une dimension aux labels pour qu'on flip bien les bons axes
                    Ynew = np.zeros(tuple([Y.shape[0]]) + tuple([1]) + Y.shape[1:])
                    Ynew[:,0,:,:] = np.copy(Y)
                    Ynew = np.flip(Ynew, axis= axe)[:,0,:,:].astype(np.uint16)
                    save_imagej_tiff(labels_output+"/"+newname, Ynew, scale, units)
            
            # on sauvegarde les données modifiées
            save_imagej_tiff4d(output_dir+"/"+newname, X, scale, units)
            
        print(i, "/", len(files), )
            
    return None    

In [12]:
############# Création des patches d'entrainement. p : numéro de patch
#inp_tif = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_noyaux"
#div_fichier = "C:/Users/bioAMD/Desktop/Nathan/Tracking/070418a_divisions.txt"
#
#output_dir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/train"
#dir_gauss = "C:/Users/bioAMD/Desktop/Nathan/Tracking/patch_gaussiens"
#
#patch_size= (32,64,64)
#nb_patches = 20
#scaling = tuple(np.float32([1.34,1.34,1.34]))
#
#indices = save_samples_patch("070418a",inp_tif, dir_gauss, output_dir, scaling, nb_patches, patch_size, div_fichier,
#              tstart=2, tstop=101, tstep=1, ch=9, demistep=True, troispas=True, nb_forward=2)

In [13]:
########### On augmente les données en faisant des rotations aux patch

##outputs
#output_dir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/augmentation/train/train_samples"
#labels_output = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/augmentation/train/train_labels"
#
## inputs
#tif_dir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/train/train_samples"
#labels_dir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/train/train_labels"
#
#scale = np.float32([1.367,1.367,1.37])
#units='um'
#
#axes_flipped = [None, 0, 1, 2, (2,0), (3,0), (2,1), (3,1), (3,2,1)]
#
#data_rotations(tif_dir, output_dir, axes_flipped, scale, units, labels_dir, labels_output)

In [14]:
######## On met les patch vides dans le stock (inutiles)
#dir_train = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/train"
#dir_stock = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/stock"
#
#samples = os.listdir(dir_train+"/train_samples")
#labels = os.listdir(dir_train+"/train_labels")
#
#for i, label in enumerate(labels) :
#    # on retire les numéros de pixels impaires (la ou il n'y a rien)
#    if int(label[13:15]) % 2 == 1 :
#        shutil.move(dir_train+"/train_labels/"+label, dir_stock+"/train_labels/"+label)
#        shutil.move(dir_train+"/train_samples/"+samples[i], dir_stock+"/train_samples/"+samples[i])

In [15]:
#help(save_imagej_tiff)

In [16]:
# dir1 = "C:/Users/bioAMD/Desktop/Nathan/VTKdata070418/stock_noyaux"
# dir2 = "C:/Users/bioAMD/Desktop/Nathan/Tracking/saved_predictions/pred_modele_divisions4"

# output = "C:/Users/bioAMD/Desktop/Nathan/Tracking/saved_predictions/pred_modele_divisions4"

# files = os.listdir(dir2)

# scale = np.float32([1.367,1.367,1.37])

# for i in range(len(files)-1) :
#     if files[i][-4:] != ".tif" or files[i][9:12] == '006':
#         continue
        
#     x = imread(dir2+"/"+files[i])
#     y = imread(dir2+"/"+files[i+1])
#     print(np.sum(x == y), np.size(x), np.size(y))

In [17]:
######### On crée des données de test
#date = "070418a"
#nb_forward = 2
#ch = 9
#
#tstart = 316
#tstop = 500
#tstep = 1
#
#dir_tif = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_noyaux"
#test_dir = "C:/Users/bioAMD/Desktop/Nathan/Tracking/dataDivisions/test"
#
#array_size = (104,512,512)
#
#files = sort_timesteps(date,dir_tif)
## on détermine l'indice correspondant à t = tstart
#indt = -1
#for i, file in enumerate(files) :
#    s = 0
#    while file[len(date)+2+s] != "_" :
#        s += 1
#    t = int(file[len(date)+2:len(date)+2+s])
#    if t == tstart :
#        indt = i
#        break
#assert indt >= 0, "Pas de temps {} non trouvé.".format(tstart)
#
#for t in range(tstart, tstop, tstep) :
#    
#    # fichier et matrice à sauvegarder
#    sample_file = test_dir + "/test_samples5/" + date + "_t"+str(t).zfill(3)+"_ch"+str(ch).zfill(2)+".tif"
#    image = np.zeros( tuple([array_size[0]]) + tuple([2*nb_forward+1]) + array_size[1:])
#    
#    print(2*(nb_forward + indt), len(files), "t=", t)
#    
#    # on sauvegarde les 2*nb_forward + 1 pas de temps sur le deuxième axe
#    for i in range(2*nb_forward+1) :
#        nomFichier = dir_tif + "/" +files[2*(i+indt-nb_forward+1)]
#        image[:,i,:,:] = imread(nomFichier)[:,0,:,:]
#        print(nomFichier)
#        
#    save_imagej_tiff4d(sample_file, image.astype(np.uint16), scale=(1.367,1.367,1.37), units="um")
#    
#    # pas de temps suivant
#    indt += tstep